In [1]:
import pandas as pd
from pandasql import sqldf

In [3]:
df = pd.read_excel("../input/Adops & Data Scientist Sample Data.xlsx", sheet_name = "Q1 Analytics")

In [5]:
df.head()

,ts,user_id,country_id,site_id
0,2019-02-01 00:01:24,LC36FC,TL6,N0OTG
1,2019-02-01 00:10:19,LC39B6,TL6,N0OTG
2,2019-02-01 00:21:50,LC3500,TL6,N0OTG
3,2019-02-01 00:22:50,LC374F,TL6,N0OTG
4,2019-02-01 00:23:44,LCC1C3,TL6,QGO3G


In [6]:
df.isna().sum()

ts            0
user_id       0
country_id    0
site_id       0
dtype: int64

In [8]:
df.describe()

,ts,user_id,country_id,site_id
count,3553,3553,3553,3553
unique,3538,1916,7,8
top,2019-02-03 20:00:34,LC3A59,TL6,5NPAU
freq,3,149,1449,1614


In [9]:
df.dtypes

ts            object
user_id       object
country_id    object
site_id       object
dtype: object

In [11]:
pysqldf = lambda q: sqldf(q, globals())

In [13]:
q = """
        SELECT *
        FROM df
"""

print(type(pysqldf(q)))

pysqldf(q).head()

<class 'pandas.core.frame.DataFrame'>


,ts,user_id,country_id,site_id
0,2019-02-01 00:01:24,LC36FC,TL6,N0OTG
1,2019-02-01 00:10:19,LC39B6,TL6,N0OTG
2,2019-02-01 00:21:50,LC3500,TL6,N0OTG
3,2019-02-01 00:22:50,LC374F,TL6,N0OTG
4,2019-02-01 00:23:44,LCC1C3,TL6,QGO3G


In [63]:
## Consider only the rows with country_id = "BDV" (there are 844 such rows). 
## For each site_id, we can compute the number of unique user_id's found in these 844 rows. 
## Which site_id has the largest number of unique users? And what's the number?


# KY: I usually add total_visit to calculate visit per user later though I did not calculate for brevity purpose now
q = """
        SELECT country_id, site_id, COUNT(DISTINCT user_id) distinct_user_count, COUNT(*) total_visit
        FROM df
        WHERE country_id = 'BDV'
        GROUP BY site_id
        ORDER BY distinct_user_count DESC
"""


pysqldf(q).head()

<class 'pandas.core.frame.DataFrame'>


,country_id,site_id,distinct_user_count,total_visit
0,BDV,5NPAU,544,717
1,BDV,N0OTG,90,122
2,BDV,3POLC,2,5


In [62]:
## Between 2019-02-03 00:00:00 and 2019-02-04 23:59:59, there are four users who visited a certain site more than 10 times. 
## Find these four users & which sites they (each) visited more than 10 times. 
## (Simply provides four triples in the form (user_id, site_id, number of visits) in the box below.)


q = """
        SELECT user_id, site_id, count(*) visit
        FROM df
        WHERE ts >= '2019-02-03 00:00:00' AND ts <= '2019-02-04 23:59:59'
        GROUP BY user_id, site_id
        HAVING  count(*) > 10
        ORDER BY visit
        
    """

pysqldf(q)

,user_id,site_id,visit
0,LC3C7E,3POLC,15
1,LC3C9D,N0OTG,17
2,LC06C3,N0OTG,25
3,LC3A59,N0OTG,26


In [80]:
## For each site, compute the unique number of users whose last visit (found in the original data set) was to that site. 
## For instance, user "LC3561"'s last visit is to "N0OTG" based on timestamp data. 
## Based on this measure, what are top three sites?
## (hint: site "3POLC" is ranked at 5th with 28 users whose last visit in the data set was to 3POLC; 
## simply provide three pairs in the form (site_id, number of users).)

q = """ 
        SELECT site_id, COUNT(*) number_of_user
        FROM
           (SELECT user_id, site_id, MAX(ts) last_visit
            FROM df
            GROUP BY user_id)
        GROUP BY site_id
        ORDER BY number_of_user DESC
        LIMIT 3

        
    """

pysqldf(q)

,site_id,number_of_user
0,5NPAU,992
1,N0OTG,561
2,QGO3G,289


In [76]:
## Validation question's explanation

q = """ 
        SELECT user_id, site_id, MAX(ts) last_visit
        FROM df
        WHERE user_id = 'LC3561'
        GROUP BY user_id
 
    
    """

pysqldf(q)

,user_id,site_id,last_visit
0,LC3561,N0OTG,2019-02-07 23:44:34


In [109]:
## For each user, determine the first site he/she visited and the last site he/she visited based on the timestamp data. 
## Compute the number of users whose first/last visits are to the same website. What is the number?

q = """ 
        SELECT count(*) user_count
        FROM
           (SELECT user_id, site_id, MAX(ts) last_visit
           FROM df
           GROUP BY user_id) t1
        INNER JOIN
           (SELECT user_id, site_id, MIN(ts) first_visit
            FROM df
            GROUP BY user_id
           ) t2
        ON t1.user_id = t2.user_id
        WHERE t1.site_id = t2.site_id
           AND t1.last_visit <> t2.first_visit
 
    
    """

## If a user visited once, which exist such as LC00C3, LC01C3, LC05C3, she/he should be excluded since there is only first visit

## Conditions in WHERE statement can be writting in ON condition as well.

pysqldf(q)

,user_count
0,409


In [123]:
q = """ 
        SELECT t1.*, t2.*
        FROM
           (SELECT user_id, site_id, MAX(ts) last_visit
           FROM df
           GROUP BY user_id) t1
        INNER JOIN
           (SELECT user_id, site_id, MIN(ts) first_visit
            FROM df
            GROUP BY user_id
           ) t2
        ON t1.user_id = t2.user_id
        WHERE t1.site_id = t2.site_id     
    
    """


len(pysqldf(q))

1670

In [117]:
## If a user visited once, she/he should be excluded

q = """ 
        SELECT *
        FROM df
        WHERE user_id = 'LC00C3' OR user_id = 'LC01C3' OR user_id = 'LC05C3'
    
    """


pysqldf(q)

,ts,user_id,country_id,site_id
0,2019-02-02 14:14:44,LC05C3,BDV,5NPAU
1,2019-02-03 18:52:50,LC00C3,QLT,5NPAU
2,2019-02-04 11:35:10,LC01C3,QLT,5NPAU


In [119]:
q = """ 
        SELECT user_id, count(*) visit
        FROM df
        GROUP BY user_id
        HAVING visit = 1
        
    """


len(pysqldf(q))

1261

In [120]:
1261 + 409

1670